<h2 style="color:#22198A">PROJECT INFO</h2>

<h3 style="color:green">About project-02</h3>
<p> This project is focused on informing policy makers and epidemiologists in NL about possible associations between the spread of COVID-19 and the weather, urbanization and demographic variables which in turn should contribute to reducing the spread of COVID-19.
<p><b>Contact:</b> luci@itu.dk, mdom@itu.dk, jses@itu.dk, mksi@itu.dk</p>
<p><b>Created:</b> 02. 03. 2021</p>
<p><b>Last modified:</b> 18. 03. 2021 </p>

<h2 style="color:#22198A">NOTEBOOK SETUP</h2>
<p>Before you start working with the notebook, please make sure to go through this setup to ensure smooth running. (by default, no changes should be needed if you just downloaded the repository)</p>
<h3 style="color:green">Important highlights</h3>
<ul>
<li><b>BASE_DIR:</b> This should lead to the root directory relative to the location of this notebook</li>
<li><b>SCRIPTS IMPORT:</b> All scripts are saved within one file. In the file, there are comments splitting the whole file into sections which gather scripts with similar functionality, e.g. loading data. All functions should contain a docstring, which might be useful for any troubleshooting or just knowing how the given thing was implemented. The way the scripts are imported was implemented according to <a href='# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
'>this</a> SO question. <b>Once you run the below cell, all scripts should be loaded.</b></li>
<li><b>PACKAGES USED WITHIN DIRECTORY: </b> In <b>all_scripts.py</b> you can see in the beginning all the packages used, but it is worth highlight these "not so standard" packages which you should make sure you have installed: <b>pandas, seaborn (=0.11.1), statsmodels, scipy.</b></li>
</ul>

In [ ]:
BASE_DIR = ""
USE_DEEPNOTE = False # In case you would open this notebook via Deepnote

# SCRIPTS IMPORT
import os
import sys
scripts_path = os.path.abspath(os.path.join(f'{BASE_DIR}scripts'))

if scripts_path not in sys.path:
    # Add the scripts to the path
    sys.path.append(scripts_path)
    
    # Import the needed scripts
    from all_scripts import *
    
    # Remove the added path to avoid possible future conflicts
    sys.path.remove(scripts_path)
else:
    
    # Import the needed scripts
    from all_scripts import *
    
    # Remove the added path to avoid possible future conflicts
    sys.path.remove(scripts_path)

# PLOTS COLOR SETTING - see more here: https://seaborn.pydata.org/generated/seaborn.color_palette.html#seaborn.color_palette
PLOT_COLOR_SETTINGS = sns.color_palette("flare", as_cmap=True) # We use seaborn as plotting package and we want to be consistent with the colors we use

<h2 style="color:#22198A">CONSTANTS</h2>

In [ ]:
PATH_DATA = {
    "raw": f"{BASE_DIR}data/raw/",
    "interim": f"{BASE_DIR}data/interim/",
    "processed": f"{BASE_DIR}data/processed/",
    "external": f"{BASE_DIR}data/external/"
}

FILENAME_DATA = {
    "weather": "weather.csv",
    "weather2": "weather2.csv",
    "corona": "nl_corona.csv",
    "metadata": "nl_metadata.json",
    "shapefiles": "nl.geojson",
    "weather_covid": "weather_covid.csv",
    "external": "nl_external.json"
}

<h2 style="color:#22198A">LOAD RAW DATA</h2>

In [ ]:
# Load data as a pandas dataframe - as a separator, use tab
WEATHER1_RAW = load_data_pandas(f"{PATH_DATA['raw']}{FILENAME_DATA['weather']}", sep='\t')
WEATHER2_RAW = load_data_pandas(f"{PATH_DATA['raw']}{FILENAME_DATA['weather2']}", sep='\t')
NL_CORONA_RAW = load_data_pandas(f"{PATH_DATA['raw']}{FILENAME_DATA['corona']}", sep='\t')

# Load meta data from the json file
METADATA_RAW = load_json(f"{PATH_DATA['raw']}{FILENAME_DATA['metadata']}")

<h2 style="color:#22198A">Task 0: Data filtering and cleaning</h2>
<h3 style="color:green">Merge two weather datasets into one</h3>

In [ ]:
# Append second weather dataset behind the first one
WEATHER_RAW = WEATHER1_RAW.append(WEATHER2_RAW)

<h3 style="color:green">Missing values</h3>
<h4 style="color:#ff9900">WEATHER DATASET</h4>
<p>Great, we can safely work with the weather dataset.</p>

In [ ]:
# Count number of rows which have missing values
WEATHER_RAW.isnull().sum()

In [ ]:
# Summary of shape
n_rows, n_cols = WEATHER_RAW.shape
print(f"Number of rows is {n_rows} and number of columns is {n_cols}.")

In [ ]:
WEATHER2_RAW.isnull().sum()

In [ ]:
# Summary of shape
n_rows, n_cols = WEATHER2_RAW.shape
print(f"Number of rows is {n_rows} and number of columns is {n_cols}.")

<h4 style="color:#ff9900">CORONA (NL) DATASET</h4>
<p>We will need to be aware of these. We will get rid of these when merging together weather and covid dataset - see a couple of cells below.</p>

In [ ]:
# Count number of rows which have missing values
NL_CORONA_RAW.isnull().sum()

In [ ]:
# Summary of shape
n_rows, n_cols = NL_CORONA_RAW.shape
print(f"Number of rows is {n_rows} and number of columns is {n_cols}.")

<h3 style="color:green">FILTER OUT IRRELEVANT RECORDS</h3>
<h4 style="color:#ff9900">WEATHER DATASET</h4>
<ul>
<li>First, we know that column iso3166-2, by which we want to filter out NL records, does not have any missing values. Therefore, we can safely use this column to get only records relevant to the Netherlands.</li>
<li>Second, we know that iso3166-2 has following format: <b>country_code-region_code</b>. We therefore split this column into two separate columns for easier handling. Based on the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html">pandas documentation</a> we decided to use the splitting string method which accepts the delimiter, number of splits and that the splitted series should be put into separate columns. In fact, for our purpose, we only need to add "Country code".</li>
</ul>

In [ ]:
# Split iso3166-2 column into two columns to easily filter records for the Netherlands and the Netherlands regions --> there will be two more columns in DF now
county_region_codes = WEATHER_RAW['iso3166-2'].str.split('-', 1, expand=True)  # Returns a DF
WEATHER_RAW['CountryCode'] = county_region_codes[0]

# Filtering out weather data for the Netherlands
WEATHER_RAW_NL = WEATHER_RAW[WEATHER_RAW['CountryCode'] == 'NL']
WEATHER_RAW_NL.shape

<h3 style="color:green">VALUE ADJUSTMENTS</h3>
<h4 style="color:#ff9900">WEATHER DATASET</h4>
<ul>
<li><b>UV INDEX:</b> Sum operation was used to aggregate data per each day (values were measured each hour). We divide it by 24 to get an average value. Note: We are aware of the fact that this is not ideal since for example during the night UV is close to zero, but for the purpose of our analysis, it should not be a problem as long as we keep it in mind when interpretting. (as discussed with Michele during last lecture)</li>
<li><b>Temperature Above Ground:</b> We just convert it from Kelvin to Celsius by subtracting 273 - it is reported as a daily average.</li>
<li><b>RelativeHumiditySurface:</b> Stays as is - it is reported as a daily average (%).</li>
<li><b>SolarRadiation:</b> We divide that by 24 - same explanation as for UV Index (Joul per square meters).</li>
<li><b>Surfacepressure:</b> Daily sum - stays as is.</li>
<li><b>Totalprecipitation:</b> Daily sum - stays as is.</li>
<li><b>WindSpeed:</b> Stays as is - it is reported as a daily average (m/s).</li>
</ul>

In [ ]:
# UV Index
uv_adjusted = WEATHER_RAW_NL.apply(lambda row: row['UVIndex']/24, axis=1)
WEATHER_RAW_NL = WEATHER_RAW_NL.assign(**{'UVIndex': uv_adjusted})

# TEMPERATURE ABOVE GROUND - from Kelvin to Celsius
temp_adjusted = WEATHER_RAW_NL.apply(lambda row: row['TemperatureAboveGround'] - 273, axis=1)
WEATHER_RAW_NL = WEATHER_RAW_NL.assign(**{'TemperatureAboveGround': temp_adjusted})

# Solar radiation
solar_adjusted = WEATHER_RAW_NL.apply(lambda row: row['SolarRadiation']/24, axis=1)
WEATHER_RAW_NL = WEATHER_RAW_NL.assign(**{'SolarRadiation': solar_adjusted})
WEATHER_RAW_NL.shape

<h4 style="color:#ff9900">CORONA DATASET</h4>
<ul>
<li><b>REGION CODE:</b> Here, we use the metadata to map the region covid code to coresponding iso-3166 code. This will then help us when merging covid and weather datasets, which will now have date and iso3166-2 common columns.</li>
</ul>

In [ ]:
# Create a dictionary where key will be covid region code and value iso3166-2_code
covid_code_to_iso = {
    country_info['covid_region_code']: country_info['iso3166-2_code'] for country_info in METADATA_RAW['country_metadata']
}

# Use this lookup dictionary to create a new column for the covid DF which will have values of iso codes for corresponding regions
covid_iso_codes = [covid_code_to_iso[str(covid_code)] for covid_code in NL_CORONA_RAW['region_code']]

# Add it as a new column to the covid data set
NL_CORONA_RAW['iso3166-2'] = covid_iso_codes
NL_CORONA_RAW.shape

<h3 style="color:green">MERGE WEATHER AND COVID DATASETS</h3>

<h4 style="color:#ff9900">MERGE DATASETS</h4>
<p>As noted earlier, the two datasets have two common columns: 'date', 'iso3166-2'. We used inner join which takes the two common columns, finds the intersection between two dataframes based on these two columns and returns a merged dataframe. In total numbers before merge:</p>
<ul>
<li>
WEATHER DATASET - NL ONLY: In total there are 4500 records which are from 13.2.2020 to 21.2.2021. There are 10 columns in total.
</li>
<li>
CORONA DATASET - NL ONLY: In total there are 4344 records which are from 27.2.2020 to 22.2.2021. There are 10 columns in total.
</li>
</ul>
<p>This means the difference between two datasets is 14 days. Since there are 12 regions, it means 12 records per day, which in total means 12*14 = 168 records. In other words, by this merge, we assume to lose 168 records. This is due to our decision to make sure that from both datasets there are records for both given date and given region. After merge, we have got 4332 records, which means 4332/12 = 361. This corresponds to the fact that data starts from 27.2.2020 and ends on 21.2.2021. Finally, the total number of column before merge was 10 + 10 = 20, and we know two columns were same, which corresponds to after merge number of columns which is 18.</p>

In [ ]:
# Merge covid and weather data set
WEATHER_COVID = WEATHER_RAW_NL.merge(NL_CORONA_RAW, how='inner', on=['date', 'iso3166-2'])
WEATHER_COVID.shape

<h4 style="color:#ff9900">DROP ROWS: missing values in corona dataset</h4>
<p>First, we want to identify the problematic rows.</p>

In [ ]:
WEATHER_COVID.isnull().sum()

<p>Let's examine during which period these missing values occur</p>

In [ ]:
for column in WEATHER_COVID.columns:
    missing_rec_mask = WEATHER_COVID[column].isnull() # Should return TRUE or FALSE series
    missing_records = WEATHER_COVID[missing_rec_mask] # Should return DF
    if missing_records.shape[0] > 0:
        print(f'Column name: {column.capitalize()}')
        print(f'The records start on {missing_records["date"].iloc[0]} and end on {missing_records["date"].iloc[-1]}')
        print()

<p>Clearly, the problematic rows are within the dates 2020-02-27 and 2020-03-13 which corresponds to 16 days. (192 records) Let's examine cumulative measures on 2020-03-14.</p>

In [ ]:
for column in ['deceased_cumulative', 'hospitalized_cumulative', 'confirmed_cumulative']:
    mask_date = WEATHER_COVID['date'] == '2020-03-14'
    print(f'Column name: {column.capitalize()}')
    print(f'Value as of 2020-03-14: {WEATHER_COVID[mask_date][column].iloc[0]}')
    print()

<p>Altough first case in NL was on 27th of February, serious measures were deployed by governement around 12th of March according to <a href='https://en.wikipedia.org/wiki/COVID-19_pandemic_in_the_Netherlands'>this</a> Wikipedia article. Furthermore, according to above values, there were not many cases yet (this actually contradicts the Wikipedia article which claims there were 321 cases on March 9). Given these inputs, we made a decision that for the purpose of our analysis, we will drop these records.</p>

In [ ]:
WEATHER_COVID = WEATHER_COVID[WEATHER_COVID['hospitalized_addition'].notnull()]
WEATHER_COVID.shape

<h4 style="color:#ff9900">DROP ROWS: negative values in confirmed, hospitalized cases</h4>
<p>Finally, we want to validate that all corona measures are non-negative. Let's examine this.</p>

In [ ]:
for column in WEATHER_COVID.columns[12:]:
    negative_vals_mask = WEATHER_COVID[column] < 0  # Returns series of booleans
    negative_vals_df = WEATHER_COVID[negative_vals_mask]  # Returns a dataframe
    if negative_vals_df.shape[0] > 0:
        print(f'This column has negative values: {column}') 
        print(f'There are {negative_vals_df.shape[0]} such records.')
        print()

<p>We looked for a reason for this within the github repository where the dataset is from, but we were only able to find the following <a href='https://github.com/J535D165/CoronaWatchNL/issues/180'>open issue</a>, which has been unanswered. Let's see whether there is any pattern when these values occur.</p>

In [ ]:
# Setup figure
sns.set(rc={'figure.figsize':(30,8)})
fig_overview, axs = plt.subplots(1, 3);
fig_overview.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=None, hspace=1.2)

index = 0
columns = ['confirmed_addition', 'hospitalized_addition', 'deceased_addition']
for ax in axs:

    # Get data
    column = columns[index]
    negative_vals_mask = WEATHER_COVID[column] < 0  # Returns series of booleans
    negative_vals_df = WEATHER_COVID[negative_vals_mask]  # Returns a dataframe
    Xs = negative_vals_df['date']
    Ys = negative_vals_df[column]

    # Plot the data
    sns.scatterplot(x=Xs, y=Ys, ax=ax, palette=sns.color_palette("flare"));

    ax.set_title(f'Negative values - {column}', size=14)

    # Set xticks
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(negative_vals_df['date'].unique(), rotation=45, ha='right');

    index += 1

<p>Looking purely on the dates, there is no clear pattern, so our assumption is that this might be the way to correct values. Let's see the total of these numbers.</p>

In [ ]:
columns = ['confirmed_addition', 'hospitalized_addition', 'deceased_addition']
for column in columns:
    negative_vals_mask = WEATHER_COVID[column] < 0  # Returns series of booleans
    negative_vals_df = WEATHER_COVID[negative_vals_mask]  # Returns a dataframe
    print(f'Column name: {column}')
    print(f'The total sum of given column is: {WEATHER_COVID[column].sum()}')
    print(f'The total sum is of negative values is: {negative_vals_df[column].sum()}, which is {round(abs(negative_vals_df[column].sum())/WEATHER_COVID[column].sum(), 5)*100} % of all values.')
    print()

<p>Given the small percentage and also the fact that there is not, at least according to our knowledge, a way how to find the values for the given day, or more precisely there might be a way, but for the scope of this project, it would not make much difference, therefore we decided to drop these records.</p>

In [ ]:
columns = ['confirmed_addition', 'hospitalized_addition', 'deceased_addition']
for column in columns:
    non_negative_mask = WEATHER_COVID[column] >= 0  # Returns series of booleans
    WEATHER_COVID = WEATHER_COVID[non_negative_mask]
WEATHER_COVID.shape

<h4 style="color:#ff9900">DROP COLUMNS</h4>
<p>As part of cleaning our dataset, we dropped irrelevant columns for our analysis.</p>

In [ ]:
# Get rid of non-neccesary columns
WEATHER_COVID = WEATHER_COVID.drop(labels=['region_code', 'CountryCode', 'deceased_cumulative', 'confirmed_cumulative', 'hospitalized_cumulative'], axis='columns')
WEATHER_COVID.shape

<h3 style="color:green">PARSE DATE</h3>
<ul>
<li>We first parse the string which has following structure "%Y-%m-%d". This allows to use strings method to parse it into needed parts and save it as an int.</li>
<li>Second, we get the week number by turning date column records into timestamps objects and the accessing its week attribute.</li>
</ul>

In [ ]:
# date_parsed = df with following columns: Year, month, date
date_parsed = WEATHER_COVID['date'].str.split('-', 2, expand=True)
WEATHER_COVID['Year'] = date_parsed[0].astype(int)
WEATHER_COVID['Month'] = date_parsed[1].astype(int)
WEATHER_COVID['Day'] = date_parsed[2].astype(int)

# Get week number
date_modified = pd.to_datetime(WEATHER_COVID['date'], format="%Y-%m-%d", origin="unix") # Strings converted to pandas timestamp object
WEATHER_COVID['Week'] = date_modified.dt.isocalendar().week.astype(int) # Use the timestamp object to access a week number

<h3 style="color:green">META DATA INTEGRATION</h3>
<p>In this part we want to integrate the population size for each region which will be useful in our further analysis.</p>

In [ ]:
# Get a mapping
map_isocode_population = {'population_data': {}}
map_isocode_population['population_data'] = {region_info['iso3166-2_code']: region_info['population'] for region_info in METADATA_RAW['country_metadata']}

# Integrate the population to dataset
WEATHER_COVID['population'] = WEATHER_COVID['iso3166-2'].map(map_isocode_population['population_data'])

# Compute covid cases per capita
WEATHER_COVID['cases_per_capita'] = WEATHER_COVID['confirmed_addition']/WEATHER_COVID['population']

# Compute covid cases per 100k
WEATHER_COVID['cases_per_100k'] = WEATHER_COVID['confirmed_addition']/WEATHER_COVID['population']*10**5

<h3 style="color:green">DATA EXPLORATION</h3>

<h4 style="color:#ff9900">GEOGRAPHIC PERSPECTIVE</h4>


In [ ]:
# Set which aggregation function you want to apply (pd.DataFrame.min, pd.DataFrame.mean, pd.DataFrame.median, pd.DataFrame.max)
agg_func = pd.DataFrame.median

# Create the DF report
cols_to_exclude = ['date', 'iso3166-2', 'Year', 'Month', 'Day', 'Week', 'region_name']
num_cols = [col_name for col_name in WEATHER_COVID.columns if col_name not in cols_to_exclude]
aggregated_numeric_by_reg = WEATHER_COVID.groupby(['region_name'], as_index=False)[num_cols].apply(agg_func)
aggregated_numeric_by_reg

<h4 style="color:#ff9900">TIME PERSPECTIVE</h4>


In [ ]:
# Create the DF report
cols_to_exclude = ['date', 'iso3166-2', 'Year', 'Month', 'Day', 'Week', 'region_name']
num_cols = [col_name for col_name in WEATHER_COVID.columns if col_name not in cols_to_exclude]
aggregated_numeric_by_date = WEATHER_COVID.groupby(['date'], as_index=False)[num_cols].apply(pd.DataFrame.mean)
aggregated_numeric_by_date

<h4 style="color:#ff9900">GENERAL PERSPECTIVE</h4>

In [ ]:
# Summary of missing values
WEATHER_COVID.isnull().sum()

In [ ]:
# Summary of shape
n_rows, n_cols = WEATHER_COVID.shape
print(f"Number of rows is {n_rows} and number of columns is {n_cols}.")

<h3 style="color:green">SAVE THE CLEAN DATA TO INTERIM</h3>

In [ ]:
save_df_to_csv(WEATHER_COVID, f"{PATH_DATA['interim']}{FILENAME_DATA['weather_covid']}")

<h2 style="color:#22198A">LOAD CLEAN DATA</h2>

In [ ]:
CLEAN_DATA = load_data_pandas(f"{PATH_DATA['interim']}{FILENAME_DATA['weather_covid']}", sep=',')
METADATA = load_json(f"{PATH_DATA['raw']}{FILENAME_DATA['metadata']}")

<h2 style="color:#22198A">Task 1: Single variable analysis</h2>
<p>Report key statistics for the relevant variables you selected for your analysis: <b>Confirmed addition, UV INDEX</b>. Do so for:
<br>
1) the different country regions
<br>
2) in different levels of aggregation (daily, weekly, monthly).
</p>

<h3 style="color:green">GEOGRAPHIC PERSPECTIVE</h3>

<h4 style="color:#ff9900">OVERVIEW OF CORONA IMPACT BY REGION</h4>

In [ ]:
# Setup figure
sns.set(rc={'figure.figsize':(20,8)})
fig_overview, axs = plt.subplots(1, 3);
fig_overview.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=None, hspace=1.2)

index = 0
single_var_sel_cols = ['confirmed_addition', 'hospitalized_addition', 'deceased_addition']
for ax in axs:

    # Get column name
    column_name = single_var_sel_cols[index]

    # Calculate total number of cases per region
    region_df = CLEAN_DATA.groupby(['iso3166-2'], as_index=False)[column_name].sum()

    # Incorporate population size for each dataframe
    region_population_name = {reg_info['iso3166-2_code']: (reg_info['population'], reg_info['iso3166-2_name_en']) for reg_info in METADATA['country_metadata']}
    region_df['population_size'] = [region_population_name[reg_code][0] for reg_code in region_df['iso3166-2']]

    # Add also corresponding region name
    region_df['region_name'] = [region_population_name[reg_code][1] for reg_code in region_df['iso3166-2']]

    # Calculate cases per capita
    region_df['cases_per_capita'] = region_df[column_name]/region_df['population_size']

    # Sort the df
    region_df = region_df.sort_values('cases_per_capita', ascending=False)

    # Plot
    sns.barplot(x='region_name', y='cases_per_capita', data=region_df, ax=ax, palette=sns.color_palette("flare", n_colors=len(region_df['region_name'])))

    # Set labels and title
    if index == 0:
        ax.set_ylabel('Frequency per capita', size=13)
    else:
        ax.set_ylabel('')
    
    ax.set_xlabel('')

    ax.set_title(f'Total {" ".join(column_name.split("_")).capitalize()} per capita\nfrom 2020-03-14 to 2021-02-21', fontweight="bold", fontsize=14)

    # Set xticks
    ax.set_xticklabels(region_df['region_name'], rotation=45, ha='right')

    index += 1

# Create a figure
fig_overview.savefig(f'{BASE_DIR}figures/figure1.png', orientation='landscape')

<h4 style="color:#ff9900">CONFIRMED CASES - REGION BY REGION</h4>

In [ ]:
# Setup figure
sns.set(rc={'figure.figsize':(20,10)})
fig, axs = plt.subplots(4, 3);
fig.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=None, hspace=2)

# Plot the data
index = 0
regions = CLEAN_DATA['region_name'].unique() # Should return 12 regions
for row in axs:
    for ax in row:
        # Get aggregation by Month for the given region
        region_mask = CLEAN_DATA['region_name'] == regions[index]
        reg_month_aggregation = CLEAN_DATA[region_mask].groupby(['Year', 'Month'], as_index=False)['confirmed_addition'].sum()

        # Combine year and month column into one
        year_month_df = pd.concat([reg_month_aggregation['Year'].astype(str), reg_month_aggregation['Month'].astype(str)], axis=1)
        reg_month_aggregation['Year - Month'] = year_month_df.agg('-'.join, axis=1)

        # Get population size for the given region
        region_population_name = {reg_info['iso3166-2_code']: (reg_info['population'], reg_info['iso3166-2_name_en']) for reg_info in METADATA['country_metadata']}
        region_iso_code = CLEAN_DATA[region_mask]['iso3166-2'].iloc[0] # Returns first item from pandas series
        population_size = region_population_name[region_iso_code][0]

        # Calculate cases per capita and add it as a new column
        reg_month_aggregation = reg_month_aggregation.assign(**{'cases_per_capita': reg_month_aggregation['confirmed_addition']/population_size})

        # Plot the data to given ax
        sns.lineplot(x='Year - Month', y='cases_per_capita', data=reg_month_aggregation, color="#22198A", palette=PLOT_COLOR_SETTINGS, ax=ax)


        # Add labels
        if index in [9, 10, 11]:
            ax.set_xlabel("Year - Month", size=13);
        else:
            ax.set_xlabel("");
        
        if index in [0, 3, 6, 9]:
            ax.set_ylabel("Confirmed\ncases per capita", size = 12);
        else:
            ax.set_ylabel("");
        ax.set_title(label=f"{regions[index]}", size=14, weight="bold", color="green");

        # Set y-limit
        ax.set_ylim(0, 0.03)

        # Adjust xticks
        old_xticks_monthly = ax.get_xticks()
        new_xticks = [old_xticks_monthly[i] for i in range(0, len(old_xticks_monthly), 1)]
        new_xticks_labels = reg_month_aggregation['Year - Month'].unique()
        new_labels = [new_xticks_labels[i] for i in range(0, len(old_xticks_monthly), 1)]
        ax.set_xticks(new_xticks);
        ax.set_xticklabels(new_labels, rotation=45);

        index += 1

# Set figure title
fig.suptitle('Number of confirmed cases\nper capita per given month (2020-03 - 2021-02)', fontsize=16, weight="bold", color="#22198A");

# Save the figure
fig.savefig(f'{BASE_DIR}figures/figure2.png', orientation='landscape')

<h4 style="color:#ff9900">UVINDEX - REGION BY REGION</h4>

In [ ]:
# Setup figure
sns.set(rc={'figure.figsize':(20,10)})
fig, axs = plt.subplots(4, 3);
fig.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=None, hspace=2)

# Plot the data
index = 0
regions = CLEAN_DATA['region_name'].unique() # Should return 12 regions
for row in axs:
    for ax in row:
        # Get aggregation by Month for the given region
        region_mask = CLEAN_DATA['region_name'] == regions[index]
        reg_month_aggregation = CLEAN_DATA[region_mask].groupby(['Year', 'Month'], as_index=False)['UVIndex'].mean()

        # Combine year and month column into one
        year_month_df = pd.concat([reg_month_aggregation['Year'].astype(str), reg_month_aggregation['Month'].astype(str)], axis=1)
        reg_month_aggregation['Year - Month'] = year_month_df.agg('-'.join, axis=1)
        
        # Plot the data to given ax
        sns.lineplot(x='Year - Month', y='UVIndex', data=reg_month_aggregation, color="#22198A", palette=PLOT_COLOR_SETTINGS, ax=ax)

        # Add labels
        if index in [9, 10, 11]:
            ax.set_xlabel("Year - Month", size=13);
        else:
            ax.set_xlabel("");
        
        if index in [0, 3, 6, 9]:
            ax.set_ylabel("Mean of\nUVindex", size = 13);
        else:
            ax.set_ylabel("");
        ax.set_title(label=f"{regions[index]}", size=14, weight="bold", color="green");

        # Set y-limit
        ax.set_ylim(0, 2.5)

        # Adjust xticks
        old_xticks_monthly = ax.get_xticks()
        new_xticks = [old_xticks_monthly[i] for i in range(0, len(old_xticks_monthly), 1)]
        new_xticks_labels = reg_month_aggregation['Year - Month'].unique()
        new_labels = [new_xticks_labels[i] for i in range(0, len(old_xticks_monthly), 1)]
        ax.set_xticks(new_xticks);
        ax.set_xticklabels(new_labels, rotation=45);

        index += 1

# Set figure title
fig.suptitle('Mean of UV index\nper given month (2020-03 - 2021-02)', fontsize=16, weight="bold", color="#22198A");

# Save figure
fig.savefig(f'{BASE_DIR}figures/figure3.png', orientation='landscape')

<h3 style="color:green">DAILY PERSPECTIVE</h3>
<p>The thick line represents the mean of values from all regions, the background transparent color then represents standard deviation.</p>

<h4 style="color:#ff9900">DAILY - CONFIRMED CASES</h4>

In [ ]:
CLEAN_DATA['confirmed_addition'].describe()

<h4 style="color:#ff9900">DAILY - UV Index</h4>

In [ ]:
CLEAN_DATA['UVIndex'].describe()

<h4 style="color:#ff9900">DAILY - ONE FIGURE</h4>

In [ ]:
# Set up figure
sns.set(rc={'figure.figsize':(20,10)})
fig, axs = plt.subplots(2, 1);

# Deal with dates properly
fig.autofmt_xdate()

# Plot the data
index = 0
for ax in axs:

    # Plot1
    if index == 0:

        # Plot the values (ci = standard deviation, hue = which categories to highlight)
        sns.lineplot(x=date_modified, y='confirmed_addition', data=CLEAN_DATA, palette=PLOT_COLOR_SETTINGS, hue='Year', ax=ax, ci='sd', estimator=pd.DataFrame.mean)

        # Add labels
        ax.set(xlabel="Year - Month", ylabel="Mean and SD of\nconfirmed cases");
        ax.set_title(label="Mean and SD of confirmed cases per given day", size=15, weight="bold");

        # Adjust legend
        ax.legend(title='Year', loc='upper right', labels=[2020, 2021]);
    
    # Plot 2
    elif index == 1:

        # Plot the values (ci = standard deviation, hue = which categories to highlight)
        sns.lineplot(x=date_modified, y='UVIndex', data=CLEAN_DATA, palette=PLOT_COLOR_SETTINGS, hue='Year', ax=ax, ci='sd', estimator=pd.DataFrame.mean)

        # Add labels
        ax.set(xlabel="Year-Month", ylabel="Mean and SD of\nadjusted UVIndex");
        ax.set_title(label="Mean and SD of adjusted UVIndex per given day", size=15, weight="bold");

        # Adjust legend
        ax.legend(title='Year', loc='upper right', labels=[2020, 2021]);
    
    index += 1

# Save figure
fig.savefig(f'{BASE_DIR}figures/figure4.png', orientation='landscape')

<h3 style="color:green">WEEKLY PERSPECTIVE</h3>
<p>The thick line represents the mean of values from all regions, the background transparent color then represents standard deviation.</p>

<h4 style="color:#ff9900">WEEKLY - CONFIRMED CASES</h4>

In [ ]:
# Get aggregation by week
week_aggregation = CLEAN_DATA.groupby(['Year', 'Week', 'region_name'], as_index=False)['confirmed_addition'].sum()

# Combine year and week column into one
week_year_df = pd.concat([week_aggregation['Year'].astype(str), week_aggregation['Week'].astype(str)], axis=1)
week_aggregation['Year - Week'] = week_year_df.agg('-'.join, axis=1)

# We also need to handle exception: Last week of 2020 has 3 days in 2021 --> we therefore consider these days to be still under week "53"
# Get correct values first by summing corresponding regions together wirhin week 53
w53_y2021 = week_aggregation[week_aggregation['Year - Week'] == '2021-53']['confirmed_addition'].reset_index(drop=True)
w53_y2020 = week_aggregation[week_aggregation['Year - Week'] == '2020-53']['confirmed_addition'].reset_index(drop=True)
corrected_vals = pd.concat((w53_y2020, w53_y2021), axis=1).sum(axis=1)

# Second, use these values to replace values of year 2020 week 53
w53_y2020_mask = week_aggregation['Year - Week'] == '2020-53'
indexes_to_update = week_aggregation[w53_y2020_mask].index
for i, row_index in enumerate(indexes_to_update):
    week_aggregation.loc[row_index, 'confirmed_addition'] = corrected_vals.iloc[i]

# Third, get rid of year 2021 week 53 values
w53_y2021_mask = week_aggregation['Year - Week'] == '2021-53'
indexes_to_drop = week_aggregation[w53_y2021_mask].index
week_aggregation_cases = week_aggregation.drop(indexes_to_drop)
week_aggregation_cases['confirmed_addition'].describe()

<h4 style="color:#ff9900">WEEKLY - UV INDEX</h4>

In [ ]:
# Get aggregation by week
week_aggregation = CLEAN_DATA.groupby(['Year', 'Week', 'region_name'], as_index=False)['UVIndex'].mean()

# Combine year and week column into one
week_year_df = pd.concat([week_aggregation['Year'].astype(str), week_aggregation['Week'].astype(str)], axis=1)
week_aggregation['Year - Week'] = week_year_df.agg('-'.join, axis=1)

# We also need to handle exception: Last week of 2020 has 3 days in 2021 --> we therefore consider these days to be still under week "53" - year 2020
# Get correct values first by summing corresponding regions together within week 53
w53_y2021 = week_aggregation[week_aggregation['Year - Week'] == '2021-53']['UVIndex'].reset_index(drop=True)
w53_y2020 = week_aggregation[week_aggregation['Year - Week'] == '2020-53']['UVIndex'].reset_index(drop=True)
corrected_vals = pd.concat((w53_y2020, w53_y2021), axis=1).mean(axis=1)

# Second, use these values to replace values of year 2020 week 53
w53_y2020_mask = week_aggregation['Year - Week'] == '2020-53'
indexes_to_update = week_aggregation[w53_y2020_mask].index
for i, row_index in enumerate(indexes_to_update):
    week_aggregation.loc[row_index, 'UVIndex'] = corrected_vals.iloc[i]

# Third, get rid of year 2021 week 53 values
w53_y2021_mask = week_aggregation['Year - Week'] == '2021-53'
indexes_to_drop = week_aggregation[w53_y2021_mask].index
week_aggregation_uv = week_aggregation.drop(indexes_to_drop)
week_aggregation_uv['UVIndex'].describe()

<h4 style="color:#ff9900">WEEKLY - ONE FIGURE</h4>

In [ ]:
# Set up figure
sns.set(rc={'figure.figsize':(20,10)})
fig, axs = plt.subplots(2, 1);

# Deal with dates properly
fig.autofmt_xdate()

# Plot the data
index = 0
for ax in axs:

    # Plot1
    if index == 0:

        # Plot the values (ci = standard deviation, hue = which categories to highlight)
        sns.lineplot(x='Year - Week', y='confirmed_addition', data=week_aggregation_cases, palette=PLOT_COLOR_SETTINGS, hue='Year', ax=ax, ci='sd')

        # Add labels
        ax.set(xlabel="Year - Week", ylabel="Mean and SD of\nconfirmed cases");
        ax.set_title(label="Mean and SD of confirmed cases per week", size=14, weight="bold");

        # Adjust legend
        ax.legend(title='Year', loc='upper right', labels=[2020, 2021]);
    
    # Plot 2
    elif index == 1:

        # Plot the values (ci = standard deviation, hue = which categories to highlight)
        sns.lineplot(x='Year - Week', y='UVIndex', data=week_aggregation_uv, palette=PLOT_COLOR_SETTINGS, hue='Year', ax=ax, ci='sd')

        # Add labels
        ax.set(xlabel="Year - Week", ylabel="Mean and SD of\nadjusted UVIndex");
        ax.set_title(label="Mean and SD of adjusted UVIndex per given week", size=14, weight="bold");

        # Adjust legend
        ax.legend(title='Year', loc='upper right', labels=[2020, 2021]);
    
    index += 1

fig.savefig(f'{BASE_DIR}figures/figure5.png', orientation='landscape')

<h3 style="color:green">MONTHLY PERSPECTIVE</h3>
<p>The thick line represents the mean of values from all regions, the background transparent color then represents standard deviation.</p>

<h4 style="color:#ff9900">MONTHLY - CONFIRMED CASES</h4>

In [ ]:
# Get aggregation
month_aggregation_cases = CLEAN_DATA.groupby(['Year', 'Month', 'region_name'], as_index=False)['confirmed_addition'].sum()

# Combine year and month column into one
year_month_df = pd.concat([month_aggregation_cases['Year'].astype(str), month_aggregation_cases['Month'].astype(str)], axis=1)
month_aggregation_cases['Year - Month'] = year_month_df.agg('-'.join, axis=1)
month_aggregation_cases['confirmed_addition'].describe()

<h4 style="color:#ff9900">MONTHLY - UV INDEX</h4>

In [ ]:
# Get aggregation
month_aggregation_uv = CLEAN_DATA.groupby(['Year', 'Month', 'region_name'], as_index=False)['UVIndex'].mean()

# Combine year and month column into one
year_month_df = pd.concat([month_aggregation_uv['Year'].astype(str), month_aggregation_uv['Month'].astype(str)], axis=1)
month_aggregation_uv['Year - Month'] = year_month_df.agg('-'.join, axis=1)
month_aggregation_uv['UVIndex'].describe()

<h4 style="color:#ff9900">MONTHLY - ONE FIGURE</h4>

In [ ]:
# Setup figure
sns.set(rc={'figure.figsize':(20,10)})
fig, axs = plt.subplots(2, 1);

# Deal with dates properly
fig.autofmt_xdate()

# Plot the data
index = 0
for ax in axs:

    # Plot1
    if index == 0:

        # Plot the values (ci = standard deviation, hue = which categories to highlight)
        sns.lineplot(x='Year - Month', y='confirmed_addition', data=month_aggregation_cases, palette=PLOT_COLOR_SETTINGS, hue='Year', ax=ax, ci='sd')

        # Add labels
        ax.set(xlabel="Year - Month", ylabel="Mean and SD of\nconfirmed cases");
        ax.set_title(label="Mean and SD of confirmed cases per given month", size=14, weight="bold");

        # Adjust legend
        ax.legend(title='Year', loc='upper right', labels=[2020, 2021]);
    
    # Plot 2
    elif index == 1:

        # Plot the values (ci = standard deviation, hue = which categories to highlight)
        sns.lineplot(x='Year - Month', y='UVIndex', data=month_aggregation_uv, palette=PLOT_COLOR_SETTINGS, hue='Year', ax=ax, ci='sd')

        # Add labels
        ax.set(xlabel="Year - Month", ylabel="Mean and SD\nof adjusted UVIndex");
        ax.set_title(label="Mean and SD of adjusted UVIndex per given month", size=14, weight="bold");

        # Adjust legend
        ax.legend(title='Year', loc='upper right', labels=[2020, 2021]);
    
    index += 1

fig.savefig(f'{BASE_DIR}figures/figure6.png', orientation='landscape')

<h2 style="color:#22198A">Task 2: Association</h2>

<h3 style="color:green">CORRELATION TEST</h3>

<h4 style="color:#ff9900">INITIAL SETUP</h4>
<p>The goal of this section is to find out whether there is an association between the weather variables and infection rates. Please note, that we have made an assumption that the effect of the weather is immediate and sustainable as we were advised to do during the lecture for purposes of this project.</p>
<ul>
<li><b>WEATHER VAR:</b> Selected weather variables which have been adjusted accordingly - see the task 0 for more detail on the transformation.</li>
<li><b>BONFERRONI CORRECTION:</b> The significance threshold needs to be adjusted with respect to the number of experiments we run. Note that the Bonferonni correction is a conservative approach, which is on purpose, since we want to make sure our claims about the results are as certain as possible.</li>
</ul>

In [ ]:
# Weather variables
WEATHER_VAR = ['RelativeHumiditySurface', 'SolarRadiation','Surfacepressure', 'TemperatureAboveGround', 'Totalprecipitation', 'UVIndex', 'WindSpeed'] 

# Significance threshold corrected by: Bonferonni correction
SIGNIFICANCE_THRESHOLD = 0.01 / (len(WEATHER_VAR))

<h4 style="color:#ff9900">DISTRIBUTION INVESTIGATION</h4>
<p>We use Q-Q plots to find out whether the variables we want to investigate follow a normal distribution or not. </p>

In [ ]:
sns.set(rc={'figure.figsize':(20,10)})
fig, axs = plt.subplots(2, 4);

index = 0

for row in axs:
    for ax in row:
        if index < len(WEATHER_VAR):
            
            # Plot the data
            sm.qqplot(CLEAN_DATA[WEATHER_VAR[index]], stats.norm, fit=True, line='45',ax=ax, color="#22198A")
            ax.set_title(f"{WEATHER_VAR[index]}", weight="bold", size=14, color= "green");

            # Set appropriatelly the labels
            if index in [0, 4]:
                ax.set_ylabel('Sample quantiles', size = 13)
            else:
                ax.set_ylabel('')
            
            if index in [4, 5, 6, 7]:
                ax.set_xlabel('Theoretical quantiles', size = 13)
            else:
                ax.set_xlabel('')

            index += 1

        elif index == len(WEATHER_VAR):
            # Plot the data
            sm.qqplot(CLEAN_DATA['confirmed_addition'], stats.norm, fit=True, line='45',ax=ax, color="#22198A")
            ax.set_title("Q-Q plot - confirmed addition", weight="bold", size=14, color= "green");

            index += 1

fig.suptitle('Q-Q plot for selected variables', fontsize=16, weight="bold", color="#22198A");
fig.savefig(f'{BASE_DIR}figures/figure7.png', orientation='landscape')

<h4 style="color:#ff9900">SELECTED VARIABLES AGAINST CONFIRMED CASES PER CAPITA</h4>

In [ ]:
sns.set(rc={'figure.figsize':(22,10)})
fig, axs = plt.subplots(2, 4);

index = 0

for row in axs:
    for ax in row:
        if index < len(WEATHER_VAR):
            
            sns.scatterplot(x = WEATHER_VAR[index], y = 'confirmed_addition', data=CLEAN_DATA, ax=ax, hue='Year', palette=PLOT_COLOR_SETTINGS)

            if index in [0, 4]:
                ax.set_ylabel('Confimed cases', size = 13)
            else:
                ax.set_ylabel('')

            index += 1

        else:
            ax.set_axis_off()

fig.suptitle('SELECTED VARIABLES AGAINST CONFIRMED CASES', fontsize=16, weight="bold", color="#22198A");
fig.savefig(f'{BASE_DIR}figures/figure8.png', orientation='landscape')

<h4 style="color:#ff9900">SPEARMAN CORRELATION</h4>
<p>We decided to use Spearman correlation as our correlation method. The reason for this choice is that our dependent variable, i.e., <b>confirmed addition</b> does not follow a normal distribution as it can be seen from the above Q-Q plot.</p>

In [ ]:
# Set up the structure of the dictionary which will then be turned into a pandas DF
SPEARMAN_RESULT = {
    'weather_var': [],
    'correlation': [],
    'p-value': [],
    'passed_sig_thrs': []
}

# Build the dictionary
for var in WEATHER_VAR:

    # Get the results
    correlation, pvalue = spearmanr(CLEAN_DATA['confirmed_addition'], CLEAN_DATA[var])

    # Save the results
    SPEARMAN_RESULT['weather_var'].append(var)
    SPEARMAN_RESULT['correlation'].append(correlation)
    if pvalue < 10**(-5):
        SPEARMAN_RESULT['p-value'].append(0.000)
    else:
        SPEARMAN_RESULT['p-value'].append(pvalue)
    SPEARMAN_RESULT['passed_sig_thrs'].append(pvalue < SIGNIFICANCE_THRESHOLD)

# Turn the dictionary into a DF
SPEARMAN_DF = pd.DataFrame.from_dict(SPEARMAN_RESULT)
SPEARMAN_DF.sort_values('correlation')

<h3 style="color:green">LOG MULTIVARIATE REGRESSION</h3>

<p>Given the large spread of our variable, we will use a logarithm function to transform it. This is indeed important to bare in mind when interpretting the coefficient values.</p>

In [ ]:
# Transform confirmed addition - if 0, then let it stay 0. Add a new column: log_confirmed_addition
CLEAN_DATA = CLEAN_DATA.assign(**{'log_confirmed_addition': [np.log(value) if value else 0 for value in CLEAN_DATA['confirmed_addition']]})

<p>In our regression, apart from the weather variables, we decided to control also for population since we are using log transformed number of new cases as our dependent variable.</p>

In [ ]:
# Add a constant
CLEAN_DATA_CONST = sm.add_constant(CLEAN_DATA)

# Create a new list with variables for multivariate regresssion and constant
WEATHER_VAR_MR = ['RelativeHumiditySurface', 'SolarRadiation','Surfacepressure', 'TemperatureAboveGround', 'Totalprecipitation', 'UVIndex', 'population', 'WindSpeed', 'const']

# Multivariate regression with confirmed_addition logtransformed
est = sm.OLS(CLEAN_DATA_CONST["log_confirmed_addition"], CLEAN_DATA_CONST[WEATHER_VAR_MR], hasconst = True).fit()
print(est.summary())

<h4 style="color:#ff9900">ADD CONTROL VARIABLES</h4>
<p>For our analysis, we want to control the fact that records are from a given region. Therefore, we are going to use a dummy variable for 11 regions, the region Gelderland is going to be set to be the reference region.</p>

In [ ]:
# Add the region to the list
provinces = list()

# Build the new columns in dataframe, columns will have a binary value - does record corresponds or not to the given region
for region_name in set(CLEAN_DATA_CONST["region_name"]):
    if region_name != "Gelderland":
        CLEAN_DATA_CONST[region_name] = (CLEAN_DATA_CONST["region_name"] == region_name).astype(int)
        provinces.append(region_name)

<p>We will now look at the model with dummy variables only. R-squared is quite low which means that there are not big differences between the regions, but on the other hand it is not completely insignificant, therefore we can expect it is going to help increase the accuracy of our model.</p>

In [ ]:
# Multivariate regression with confirmed_addition logtransformed and only dummy variables
est = sm.OLS(CLEAN_DATA_CONST["log_confirmed_addition"], CLEAN_DATA_CONST[provinces + ['const']] , hasconst = True).fit()
print(est.summary())

<p>Finally, we can run our final model.</p>

In [ ]:
# Add provinces as variables to the multivariate regression
WEATHER_VAR_MR.extend(provinces)

# Multivariate regression with confirmed_addition logtransformed
est = sm.OLS(CLEAN_DATA_CONST["log_confirmed_addition"], CLEAN_DATA_CONST[WEATHER_VAR_MR], hasconst = True).fit()
print(est.summary())

<h4 style="color:#ff9900">VISUALIZATION</h4>

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
ax = sns.scatterplot(x='UVIndex',y='log_confirmed_addition', data=CLEAN_DATA, hue='Year', palette=PLOT_COLOR_SETTINGS)
ax.set_title('Daily mean of UVIndex and log of confirmed cases per day\n', size=15, weight="bold");
ax.set_xlabel('Mean of UVIndex per day', size=13.5);
ax.set_ylabel('Log of daily confirmed cases', size=13.5);
ax.get_figure().savefig(f'{BASE_DIR}figures/figure9.png', orientation='landscape')

<h2 style="color:#22198A">Task 3: Map visualization</h2>
<p>Our goal in this section is to visualize cases per capita for the given region within the analysis period.</p>

<h4 style="color:#ff9900">GEODATA WRANGLING</h4>

In [ ]:
# Calculate total number of cases per region
region_df = CLEAN_DATA.groupby(['iso3166-2'], as_index=False)['confirmed_addition'].sum()

# Incorporate population size for each dataframe
region_population_name = {reg_info['iso3166-2_code']: (reg_info['population'], reg_info['iso3166-2_name_en']) for reg_info in METADATA['country_metadata']}
region_df['population_size'] = [region_population_name[reg_code][0] for reg_code in region_df['iso3166-2']]

# Add also corresponding region name
region_df['region_name'] = [region_population_name[reg_code][1] for reg_code in region_df['iso3166-2']]

# Calculate cases per capita
region_df['cases_per_capita'] = region_df['confirmed_addition']/region_df['population_size']

# Calculate cases per 1k
region_df['cases_per_1k'] = region_df['confirmed_addition']/region_df['population_size']*10**3

<h4 style="color:#ff9900">CASES PER CAPITA OVERVIEW</h4>

In [ ]:
# Create a folium map object
cases_per_capita_M = folium.Map(
    location=[52.228376475929956, 5.56858503151697],
    zoom_start=7.25,
    tiles='http://tile.stamen.com/terrain/{z}/{x}/{y}.png',
    attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.')

# Add a new layer to the map
folium.Choropleth(
    geo_data=f"{PATH_DATA['raw']}{FILENAME_DATA['shapefiles']}",
    name='Cases per capita NL',
    data=region_df,
    columns=['iso3166-2', 'cases_per_capita'],
    key_on='properties.iso_3166_2',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Cases per capita'
).add_to(cases_per_capita_M)

# Show the map
folium_show(cases_per_capita_M, use_deepnote=USE_DEEPNOTE)

In [ ]:
# Create a folium map object
cases_per_capita_M = folium.Map(
    location=[52.228376475929956, 5.56858503151697],
    zoom_start=7.25,
    tiles='http://tile.stamen.com/terrain/{z}/{x}/{y}.png',
    attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.')

# Add a new layer to the map
folium.Choropleth(
    geo_data=f"{PATH_DATA['raw']}{FILENAME_DATA['shapefiles']}",
    name='Cases per 1k -  NL',
    data=region_df,
    columns=['iso3166-2', 'cases_per_1k'],
    key_on='properties.iso_3166_2',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Cases per 1k'
).add_to(cases_per_capita_M)

# Show the map
folium_show(cases_per_capita_M, use_deepnote=USE_DEEPNOTE)

<h2 style="color:#22198A">Task 4: Open question</h2>
<p>In this section, we want to further investigate the pattern which we have seen in the previous task, i.e., northern regions and Zeeland, have lower number of cases per capita than southern regions. For this reason, we will conduct an analysis to see whether there is a stastistically signifficant association between confirmed cases per capita and:</p>
<ul>
<li>GDP per capita in given region</li>
<li>Percentage of people living in non-rural area in given region</li>
<li>Percentage of people living in very urban area in given region</li>
</ul>

<h3 style="color:green">LOAD EXTERNAL DATASET</h3>

In [ ]:
# Load the external data
external_data = load_json(f'{PATH_DATA["external"]}{FILENAME_DATA["external"]}')

# Add the new variables to the dataframe which aggregates data about regions
region_df['gdp_per_capita'] = region_df['region_name'].map(external_data['province_gdp'])
region_df['rural_pct'] = region_df['region_name'].map(external_data['province_rural_pct'])
region_df['non-rural_pct'] = [100-pct for pct in region_df['rural_pct']]
region_df['very_urban_pct'] = region_df['region_name'].map(external_data['province_very_urban_pct'])

# Define new variables
PROVINCIAL_VARIABLES = ['gdp_per_capita', 'non-rural_pct', 'very_urban_pct']

<h3 style="color:green">VISUALIZE AND EXPLORE DATA</h3>

<h4 style="color:#ff9900">Q-Q Plots</h4>
<p>Despite having a small number of points, we still decided to use Q-Q plots to get an idea, whether the variables follow a normal distribution. Below we can see that all variables seem to follow a normal distribution.</p>

In [ ]:
sns.set(rc={'figure.figsize': (22,9)})
fig, axs = plt.subplots(2, 2);
index = 0

for row in axs:
    for ax in row:
        if index < len(PROVINCIAL_VARIABLES):
            
            # Plot the data
            sm.qqplot(region_df[PROVINCIAL_VARIABLES[index]], stats.norm, fit=True, line='45',ax=ax, color="#22198A")
            ax.set_title(f"{PROVINCIAL_VARIABLES[index]}", weight="bold", size=14, color= "green");

            if index in [0, 2]:
                ax.set_ylabel('Sample quantiles', size = 13)
            else:
                ax.set_ylabel('')
            
            if index in [2, 3]:
                ax.set_xlabel('Theoretical quantiles', size = 13)
            else:
                ax.set_xlabel('')

            index += 1
        else:
            sm.qqplot(region_df['cases_per_capita'], stats.norm, fit=True, line='45',ax=ax, color="#22198A")
            ax.set_title(f"Cases per capita", weight="bold", size=14, color= "green");

fig.suptitle('Q-Q plot for provincial variables and cases for capita', fontsize=16, weight="bold", color="#22198A");
fig.savefig(f'{BASE_DIR}figures/figure10.png', orientation='landscape')

<h4 style="color:#ff9900">Confirmed cases per capita VS selected variables</h4>
<p>Looking at the actual distribution again response variable, we see that, at least in the first two plots, there seems to be a linear relationship. On the other hand, one should also bare in mind the small number of data points.</p>

In [ ]:
sns.set(rc={'figure.figsize':(20,5)})
fig, axs = plt.subplots(1, 3);

index = 0

for ax in axs:
    if index < len(PROVINCIAL_VARIABLES):
        
        # Plot the data
        sns.scatterplot(x = region_df[PROVINCIAL_VARIABLES[index]], y = 'cases_per_capita', data=region_df, ax=ax, palette=PLOT_COLOR_SETTINGS)

        if index in [0, 4]:
            ax.set_ylabel('Number of confimed cases per capita', size = 13)
        else:
            ax.set_ylabel('')

        index += 1

    else:
        ax.set_axis_off()

fig.suptitle('SELECTED VARIABLES AGAINST CONFIRMED CASES PER CAPITA', fontsize=16, weight="bold", color="#22198A");
fig.savefig(f'{BASE_DIR}figures/figure11.png', orientation='landscape')

<h4 style="color:#ff9900">Map visualization</h4>
<p>We use choropleths to visualize the selected variables on a map. We can see that the map representing percentage of people living in a non-rural area is quite similar to map from task 3, where we visualized cases per capita for given region.</p>

In [ ]:
def map_from_regions(column, use_deepnote=USE_DEEPNOTE):
    map_object = folium.Map(
        location=[52.228376475929956, 5.56858503151697],
        zoom_start=7.25,
        tiles='http://tile.stamen.com/terrain/{z}/{x}/{y}.png',
        attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.')

    # Add a new layer to the map
    folium.Choropleth(
        geo_data=f"{PATH_DATA['raw']}{FILENAME_DATA['shapefiles']}",
        name='GDP per capita NL',
        data=region_df,
        columns=['iso3166-2', column],
        key_on='properties.iso_3166_2',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name= column
    ).add_to(map_object)

    # Show the map
    return folium_show(map_object, use_deepnote=USE_DEEPNOTE)

map_from_regions('gdp_per_capita')

In [ ]:
map_from_regions('non-rural_pct')

In [ ]:
map_from_regions('very_urban_pct')

<h3 style="color:green">CORRELATION TESTs</h3>
<p>Based on our Q-Q plot and the visualized distribution of the selected variables, we decided to select Pearson correlation since it is reasonable to assume that there is a linear relationship.</p>

In [ ]:
# Significance threshold corrected by: Bonferonni correction
SIGNIFICANCE_THRESHOLD = 0.01 / (len(PROVINCIAL_VARIABLES))

<h4 style="color:#ff9900">PEARSON CORRELATION TEST</h4>

In [ ]:
# Setup the structure of the dictionary which will then be turned into a pandas DF
PEARSON_RESULT = {
    'province_var': [],
    'correlation': [],
    'p-value': [],
    'passed_sig_thrs': []
}

# Build the dictionary
for var in PROVINCIAL_VARIABLES:

    # Get the results
    correlation, pvalue = pearsonr(region_df['cases_per_capita'], region_df[var])

    # Save the results
    PEARSON_RESULT['province_var'].append(var)
    PEARSON_RESULT['correlation'].append(correlation)
    if pvalue < 10**(-5):
        PEARSON_RESULT['p-value'].append(0.000)
    else:
        PEARSON_RESULT['p-value'].append(pvalue)
    PEARSON_RESULT['passed_sig_thrs'].append(pvalue < SIGNIFICANCE_THRESHOLD)


# Turn the dictionary to DF
PEARSON_DF = pd.DataFrame.from_dict(PEARSON_RESULT)
PEARSON_DF.sort_values('correlation')

<h4 style="color:#ff9900">VISUALIZATION</h4>

In [ ]:
sns.set(rc={'figure.figsize':(15,8)})
sns_scatterplot = sns.scatterplot(x=region_df['non-rural_pct'], y=region_df['cases_per_capita'], data=region_df, hue='region_name');
sns_scatterplot.set_title("Percentage of people living in non-rural area VS cases per capita", size=15, weight="bold");
sns_scatterplot.get_figure().savefig(f'{BASE_DIR}figures/figure12.png', orientation='landscape')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c421fda8-2f73-4274-82d5-66d77a676a1e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>